In [1]:
import tensorflow as tf
import numpy as np
import sklearn.datasets
import matplotlib.pyplot as plt

C:\Users\admin\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Zad.
Zbuduj Sieć Nauronową 

  * z pięcioma ukrytymi warstwami po 100 neuronów, 
  * inicjalizacją - He initialization
  * funkcją aktywacji ELU.



In [3]:
he_init = tf.variance_scaling_initializer()

def dnn(inputs, n_hidden_layers=5, n_neurons=100, name=None,
        activation=tf.nn.elu, initializer=he_init):
    with tf.variable_scope(name, "dnn"):
        for layer in range(n_hidden_layers):
            inputs = tf.layers.dense(inputs, n_neurons, activation=activation,
                                     kernel_initializer=initializer,
                                     name="hidden%d" % (layer + 1))
        return inputs

In [5]:
n_inputs = 28 * 28 # MNIST
n_outputs = 5

tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

dnn_outputs = dnn(X)

logits = tf.layers.dense(dnn_outputs, n_outputs, kernel_initializer=he_init, name="logits")
Y_proba = tf.nn.softmax(logits, name="Y_proba")

# Zad 

Używając optymalizacji Adama aby nauczyć Siceć z poprzedniego zadani na MNIST, ale tylko na cyfrach od 0 do 4.

In [7]:

learning_rate = 0.01

xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy, name="loss")

optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss, name="training_op")

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [9]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/")

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [10]:
X_train = mnist.train.images
y_train = mnist.train.labels

X_valid = mnist.validation.images
y_valid = mnist.validation.labels

X_test = mnist.test.images
y_test = mnist.test.labels

X_train1 = X_train[y_train < 5]
y_train1 = y_train[y_train < 5]

X_valid1 = X_valid[y_valid < 5]
y_valid1 = y_valid[y_valid < 5]

X_test1 = X_test[y_test < 5]
y_test1 = y_test[y_test < 5]

In [12]:
X_train1.shape[1]

784

In [14]:
n_epochs = 20
batch_size = 200

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_index in range(X_train1.shape[1] // batch_size):
            print("\r{}%".format(100 * batch_index //  (X_train1.shape[1] // batch_size) ), end="")
            
            shuffle_index = np.random.permutation(batch_size)
            X_batch, y_batch = X_train1[shuffle_index], y_train1[shuffle_index]
            
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test1, y: y_test1})
        print(" ", epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test) 

66%  0 Train accuracy: 0.355 Test accuracy: 0.32554972
66%  1 Train accuracy: 0.755 Test accuracy: 0.71414673
66%  2 Train accuracy: 0.94 Test accuracy: 0.89569956
66%  3 Train accuracy: 0.95 Test accuracy: 0.9145748
66%  4 Train accuracy: 0.985 Test accuracy: 0.89803463
66%  5 Train accuracy: 0.995 Test accuracy: 0.91340727
66%  6 Train accuracy: 1.0 Test accuracy: 0.9128235
66%  7 Train accuracy: 1.0 Test accuracy: 0.9172991
66%  8 Train accuracy: 1.0 Test accuracy: 0.92060715
66%  9 Train accuracy: 1.0 Test accuracy: 0.92605567
66%  10 Train accuracy: 1.0 Test accuracy: 0.9291691
66%  11 Train accuracy: 1.0 Test accuracy: 0.92819613
66%  12 Train accuracy: 1.0 Test accuracy: 0.9287799
66%  13 Train accuracy: 1.0 Test accuracy: 0.9287799
66%  14 Train accuracy: 1.0 Test accuracy: 0.9291691
66%  15 Train accuracy: 1.0 Test accuracy: 0.9291691
66%  16 Train accuracy: 1.0 Test accuracy: 0.92858535
66%  17 Train accuracy: 1.0 Test accuracy: 0.9287799
66%  18 Train accuracy: 1.0 Test accu